# Sentiment Analysis
> * Positive or Negative

In [1]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re

from collections import namedtuple
from gensim.models import doc2vec
import multiprocessing
from konlpy.tag import Mecab
from konlpy.utils import pprint
import numpy as np
from ckonlpy.tag import Twitter as ctwitter
mecab = Mecab()
ct = ctwitter()

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument

In [13]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')

## 뉴스 빅데이터 현황으로부터 감정분석 문장 추출

In [3]:
newsSentence = pd.read_csv('./data/sentiment_data/merged_sentiment_data.txt',encoding='utf-8', header=None)
newsSentence.head()

,0,1
0,이젠 민주개혁의 길로 사설 이 90년대가 우리 모두에게 성취의 시대이기를 기원하고...,1
1,민주당과 공화당은 새해부터 정계개편을 적극 추진한다는 방침이며 양당의 김영삼ㆍ김종필...,1
2,"4당,지자제 연합공천 대비 파트너 탐색전 이같은 구상을 실현시키기 위해 민정당은...",1
3,90년대는 국내적으로 정치민주화와 선진국에로의 도약여부가 판가름날 것이고 한반도를 ...,1
4,"특히 노태우대통령이 3일 대국민 특별담화를 발표, 지난시대 의 완전청산을 선언한 후...",1


In [4]:
positiveWords = pd.read_csv('./data/sentiment_data/positive_sentiment_word_from_news.csv',encoding='utf-8',header=None)
negativeWords = pd.read_csv('./data/sentiment_data/negative_sentiment_word_from_news.csv', encoding='utf-8', header=None)

In [5]:
positiveWords = pd.concat([positiveWords, pd.DataFrame(len(positiveWords) * [1], columns=[1])], axis = 1)
negativeWords = pd.concat([negativeWords, pd.DataFrame(len(negativeWords) * [0], columns=[1])], axis = 1)

In [6]:
rawdata = pd.concat([newsSentence, positiveWords, negativeWords])

In [7]:
rawdata.to_csv('./data/sentiment_data/raw_data_sentiment.txt',index=False, header = False, encoding='utf-8')

## 네이버 영화 리뷰

In [8]:
def read_data(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # header
    return data

In [9]:
rating = read_data('./data/sentiment_data/ratings.txt')

## Doc2Vec 생성

In [10]:
# twitter
def tokenize1(doc):
    return ['/'.join(t) for t in ct.pos(doc)]

In [11]:
# mecab
def tokenize2(doc):
    return ['/'.join(t) for t in mecab.pos(doc)]

### tagging twitter

In [25]:
rating_doc_ct = [(tokenize1(row[1]), row[2]) for row in rating]
news_doc_ct = [(tokenize1(newsSentence.loc[idx][0]), newsSentence.loc[idx][1]) for idx in newsSentence.index]
ct_token = rating_doc_ct+news_doc_ct
pickle.dump(ct_token, open('./data/pre_data/pre_data_by_ct_for_sentiment_analysis.pickled','wb'))

In [27]:
tagged_ct = [TaggedDocument(d, [c]) for d, c in ct_token]
pickle.dump(tagged_ct, open('./data/pre_data/pre_by_ct_data_tagged_run_docs.pickled','wb'))

#### model 만들기

In [6]:
tagged_ct = pickle.load(open('./data/pre_data/pre_by_ct_data_tagged_run_docs.pickled','rb'))

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(tagged_ct, test_size=0.15, random_state=42)

In [8]:
doc_vectorizer = doc2vec.Doc2Vec(size=3000, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(train)

for epoch in range(30):
    doc_vectorizer.train(train, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002  # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha  # fix the learning rate, no decay

#To save
doc_vectorizer.save('./model/doc2vec_size3000_epoch30_by_ct.model')
pprint(doc_vectorizer.most_similar('문재인/Noun'))
pprint(doc_vectorizer.most_similar('노무현/Noun'))
pprint(doc_vectorizer.most_similar('박근혜/Noun'))

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


[('문/Noun', 0.29109200835227966),
 ('안철수/Noun', 0.27684521675109863),
 ('박근혜/Noun', 0.25886741280555725),
 ('정세균/Noun', 0.25190475583076477),
 ('손학규/Noun', 0.24621742963790894),
 ('이태근/Noun', 0.24516235291957855),
 ('박/Noun', 0.2449999749660492),
 ('김주학/Noun', 0.22950048744678497),
 ('김한길/Noun', 0.2208537608385086),
 ('추미애/Noun', 0.21908104419708252)]
[('노/Noun', 0.4200497269630432),
 ('이명박/Noun', 0.4080064594745636),
 ('박근혜/Noun', 0.3919544219970703),
 ('박/Noun', 0.3099808096885681),
 ('김영삼/Noun', 0.3044901192188263),
 ('노태우/Noun', 0.2750924825668335),
 ('全斗換/Foreign', 0.27245914936065674),
 ('盧/Foreign', 0.2671722173690796),
 ('박정희/Noun', 0.2647184133529663),
 ('김대중/Noun', 0.2624438405036926)]
[('박/Noun', 0.525599479675293),
 ('이명박/Noun', 0.4496101438999176),
 ('노무현/Noun', 0.3919544219970703),
 ('노/Noun', 0.3626362681388855),
 ('김/Noun', 0.3456989526748657),
 ('김대중/Noun', 0.3315896689891815),
 ('김영삼/Noun', 0.3314313292503357),
 ('“朴/Foreign', 0.314728707075119),
 ('金泳三/Foreign', 0.30

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if sys.path[0] == '':
C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  del sys.path[0]


* size 2000, epoch 30 으로 모델 생성
* size 3000, epoch 30 으로 모델 생성

### tagging mecab

In [18]:
rating_doc_mecab = [(tokenize2(row[1]), row[2]) for row in rating]
news_doc_mecab = [(tokenize2(newsSentence.loc[idx][0]), newsSentence.loc[idx][1]) for idx in newsSentence.index]
mecab_token = rating_doc_mecab+news_doc_mecab
pickle.dump(mecab_token, open('./data/pre_data/pre_data_by_mecab_for_sentiment_analysis.pickled','wb'))

In [19]:
tagged_mecab = [TaggedDocument(d, [c]) for d, c in mecab_token]
pickle.dump(tagged_mecab, open('./data/pre_data/pre_by_mecab_data_tagged_run_docs.pickled','wb'))

In [9]:
tagged_mecab = pickle.load(open('./data/pre_data/pre_by_mecab_data_tagged_run_docs.pickled','rb'))

#### model 만들기

In [10]:
train2, test2 = train_test_split(tagged_mecab, test_size=0.15, random_state=42)

In [11]:
doc_vectorizer2 = doc2vec.Doc2Vec(size=3000, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer2.build_vocab(train2)

for epoch in range(30):
    doc_vectorizer2.train(train2, total_examples=doc_vectorizer2.corpus_count, epochs=doc_vectorizer2.iter)
    doc_vectorizer2.alpha -= 0.002  # decrease the learning rate
    doc_vectorizer2.min_alpha = doc_vectorizer.alpha  # fix the learning rate, no decay

#To save
doc_vectorizer2.save('./model/doc2vec_size3000_epoch30_by_mecab.model')
pprint(doc_vectorizer2.most_similar('문재인/NNP'))
pprint(doc_vectorizer2.most_similar('노무현/NNP'))
pprint(doc_vectorizer2.most_similar('박근혜/NNP'))

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


[('안철수/NNP', 0.4411509037017822),
 ('문/VV+ETM', 0.43933624029159546),
 ('손학규/NNP', 0.43808022141456604),
 ('정세균/NNP', 0.42908358573913574),
 ('김한길/NNP', 0.4065040349960327),
 ('김종인/NNP', 0.400411993265152),
 ('문재/NNG', 0.361728698015213),
 ('추미애/NNP', 0.3589673936367035),
 ('전혜숙/NNP', 0.3589380383491516),
 ('한화갑/NNP', 0.35712987184524536)]
[('이명박/NNP', 0.5369610786437988),
 ('노/NNP', 0.5254369378089905),
 ('김영삼/NNP', 0.4930700957775116),
 ('박근혜/NNP', 0.47881996631622314),
 ('노/IC', 0.4319179356098175),
 ('노태우/NNP', 0.41837140917778015),
 ('최규하/NNP', 0.4099385738372803),
 ('김대중/NNP', 0.3965388536453247),
 ('이승만/NNP', 0.387555330991745),
 ('노/XPN', 0.3838627338409424)]
[('이명박/NNP', 0.5188106894493103),
 ('박/NNP', 0.506790280342102),
 ('노무현/NNP', 0.47881999611854553),
 ('노태우/NNP', 0.3853256404399872),
 ('三/SH', 0.375473290681839),
 ('최규하/NNP', 0.37304985523223877),
 ('유셴코/NNP', 0.3641968369483948),
 ('김영삼/NNP', 0.36159783601760864),
 ('룰라/IC', 0.3606945872306824),
 ('MB/SL', 0.35922878980

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if sys.path[0] == '':
C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  del sys.path[0]


* size 2000, epoch 30 으로 모델 생성
* size 3000, epoch 30 으로 모델 생성